<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap8_semantic_search_and_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install langchain==0.2.5 faiss-cpu==1.8.0 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
!pip install llama-cpp-python==0.2.78  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [6]:
import cohere
from google.colab import userdata

try:
  api_key = userdata.get("COHERE_API_KEY")
except userdata.notebook_secret.NotebookSecretError:
  print("Please store your Cohere API key in Colab secrets with the name 'COHERE_API_KEY'")
  api_key = None

if api_key:
  co = cohere.Client(api_key)
else:
  print("Cohere client could not be initialized due to missing API key.")

In [7]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# テキストを文単位に分割
texts = text.split('.')

# スペースや改行を除去
texts = [t.strip(' \n') for t in texts]

In [8]:
import numpy as np

# 埋め込みを取得
response = co.embed(
    texts=texts,
    input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

(15, 4096)


In [10]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(np.float32(embeds))

True


ValueError: input not a numpy array

In [ ]:
import pandas as pd

def search(query, number_of_results=3):

  # 1.クエリの埋め込みを取得
  query_embed = co.embed(
      texts=[query],
      input_type="search_query"
  ).embeddings[0]

  # 2.最近傍を検索
  distances, similar_item_ids = index.search(
      np.float32([query_embed]),
      number_of_results
  )

  # 3.結果を整形
  # インデックス化を楽にするため、テキストのリストをNumPyの配列に変換
  texts_np = np.array(texts)
  results = pd.DataFrame(
      data={
          "texts": texts_np[similar_item_ids[0]],
          "distance": distances[0]
      }
  )

  # 4.結果を表示して返す
  print(f"Query:'{query}'\nNearest neighbours:")
  return results

In [ ]:
query = "科学的にはどれくらい正確だった？"
results = search(query)
results

In [ ]:
import string

from rank_bm25 import MB250kapi
from sklearn.feature_extraction import _stop_words

def bm25_tokenizer(text):
  tokenized_doc = []
  for token in text.lower().split():
    token = token.strip(string.punctuation)

    if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
      tokenized_doc.append(token)
      returntokenized_doc

In [ ]:
from tqdm import tqdm

tokenized_corups = []
for passage in tqdm(texts):
  tokenized_corups.append(bm25_tokenizer(passage))

bm25 = BM250kapi(tokenized_corpus)

In [ ]:
def keyword_search(query, top_k=3, num_candidates=15):
  print("Input question:", query)

  # BM25 を用いた検索
  bm25_scores = bm25.get_scores(bm25_tokenizer(query))
  top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
  bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
  bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

  print(f"Top-3 lexical search (BM25) hits")
  for hit in bm25_hits[0:top_k]:
    print("\t{:3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

In [ ]:
keyword_search(query="how precise was the science")